# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [10]:
site = Website("https://ai.google/")
site.links

['/',
 '/',
 '/advancing-ai/why-ai/',
 '/advancing-ai/milestones',
 '/advancing-ai/research/',
 '/advancing-ai/social-impact/',
 '/advancing-ai/why-ai/',
 '/advancing-ai/milestones',
 '/advancing-ai/research/',
 '/advancing-ai/social-impact/',
 '/responsibility/principles/',
 '/responsibility/safety/',
 '/responsibility/public-policy-perspectives/',
 '/responsibility/building-for-everyone/',
 '/responsibility/principles/',
 '/responsibility/safety/',
 '/responsibility/public-policy-perspectives/',
 '/responsibility/building-for-everyone/',
 '/get-started/gemini-ecosystem/',
 '/get-started/products/',
 'https://labs.google/',
 '/get-started/for-developers/',
 '/get-started/our-models/',
 '/get-started/for-organizations/',
 '/get-started/gemini-ecosystem/',
 '/get-started/products/',
 'https://labs.google/',
 '/get-started/for-developers/',
 '/get-started/our-models/',
 '/get-started/for-organizations/',
 '/applied-ai/health/',
 '/applied-ai/science/',
 '/applied-ai/sustainability/',
 'h

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [11]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [12]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [13]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [14]:
print(get_links_user_prompt(site))

Here is the list of links on the website of https://ai.google/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/
/
/advancing-ai/why-ai/
/advancing-ai/milestones
/advancing-ai/research/
/advancing-ai/social-impact/
/advancing-ai/why-ai/
/advancing-ai/milestones
/advancing-ai/research/
/advancing-ai/social-impact/
/responsibility/principles/
/responsibility/safety/
/responsibility/public-policy-perspectives/
/responsibility/building-for-everyone/
/responsibility/principles/
/responsibility/safety/
/responsibility/public-policy-perspectives/
/responsibility/building-for-everyone/
/get-started/gemini-ecosystem/
/get-started/products/
https://labs.google/
/get-started/for-developers/
/get-started/our-models/
/get-started/for-organizations/
/get-started/gemini-ecosystem/
/get-started/products/
https://labs.google

In [16]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [17]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/deepseek-ai/DeepSeek-V3-0324',
 '/Qwen/Qwen2.5-Omni-7B',
 '/manycore-research/SpatialLM-Llama-1B',
 '/ByteDance/InfiniteYou',
 '/ds4sd/SmolDocling-256M-preview',
 '/models',
 '/spaces/ByteDance/InfiniteYou-FLUX',
 '/spaces/3DAIGC/LHM',
 '/spaces/Trudy/gemini-codrawing',
 '/spaces/tencent/Hunyuan-T1',
 '/spaces/stabilityai/stable-virtual-camera',
 '/spaces',
 '/datasets/nvidia/Llama-Nemotron-Post-Training-Dataset-v1',
 '/datasets/glaiveai/reasoning-v1-20m',
 '/datasets/FreedomIntelligence/medical-o1-reasoning-SFT',
 '/datasets/a-m-team/AM-DeepSeek-R1-Distilled-1.4M',
 '/datasets/facebook/collaborative_agent_bench',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/go

In [18]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'models page', 'url': 'https://huggingface.co/models'},
  {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'},
  {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'},
  {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [19]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [20]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
deepseek-ai/DeepSeek-V3-0324
Updated
1 day 

In [21]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [22]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [23]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'social media', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


"You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-V3-0324\nUpdated\n1 day ago\n•\n47.6k\n•\n1.91k\nQwen/Qwen2.5-Omni-7B\nUpdated\nabout 2 hours ago\n•\n16.3k\n•\n712\nmanycore-research/SpatialLM-Llama-1B\nUpdated\n7 days ago\n•\n6.85k\n•\n760\nByteDance/InfiniteYou\nUpdated\n3 days ago\n•\n448\nds4sd/SmolDocling-256M-preview\nUpdated\n5 days ago\n•\n44.8k\n•\n1.01k\nBrowse 1M+ models\nSpaces\nRunning\non\nZero\n493\n493\n

In [24]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [25]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

## Welcome to Hugging Face

**Hugging Face** is at the forefront of the artificial intelligence and machine learning revolution, dedicated to building a collaborative community that shapes the future of AI. Our platform is where the machine learning community connects, shares, and innovates—whether it's models, datasets, or applications.

### Why Choose Hugging Face?

- **Extensive Model Library:** Explore and utilize over **1 million% models** across various modalities including text, image, video, audio, and 3D.
- **Datasets for All:** Access and share more than **250,000 datasets** for your machine learning tasks.
- **Dedicated Spaces:** Run and showcase **400,000+ applications** in our dedicated spaces.
- **Open Source Collaboration:** Contribute to cutting-edge tooling and libraries like Transformers, Diffusers, and more within our vibrant community.

### Who We Serve

Hugging Face is trusted by more than **50,000 organizations**, including industry leaders such as:
- **Google**
- **Microsoft**
- **Amazon**
- **Meta**
- **Intel**
- **Grammarly**

### Our Company Culture

At Hugging Face, we foster a **collaborative and inclusive environment** where innovation thrives. Our team is not just about technology; it's about people coming together to share knowledge, support one another, and create meaningful solutions. We promote continuous learning and encourage our team members to contribute ideas that drive the company forward.

### Careers & Opportunities

We are always looking for talent! Join us and become a part of the future of AI. **Explore career opportunities** at Hugging Face to find positions that align with your passion and skills. Whether you're an engineer, researcher, or enthusiast, we welcome diverse skills and perspectives to help shape the future of AI together.

**Interested in joining us? [Find out more about current job openings.](#)**

### Join the AI Revolution

Ready to be part of something bigger? **[Sign up now](#)** to start exploring, collaborating, and innovating. Together, let's build the future of AI!

--- 

For more information, visit us at [huggingface.co](https://huggingface.co). Follow us on social media; we're active on GitHub, Twitter, LinkedIn, and more!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [26]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [27]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

---

## Company Overview

**Hugging Face** is the leading platform for the AI community focused on machine learning collaboration. With over one million models available for exploration, Hugging Face serves as a pivotal resource where individuals and organizations can create, discover, and work together on innovative ML projects. The company empowers developers, researchers, and enterprises to accelerate their machine learning endeavors through tools, datasets, and applications designed for a collaborative experience.

---

## Our Offerings

- **Models**: Access a wealth of machine learning models spanning various applications from text to images and beyond.
- **Datasets**: Browse over 250,000 datasets for all your machine learning needs.
- **Spaces**: Explore custom applications powered by Hugging Face technology that can run seamlessly on the platform.
- **Enterprise Solutions**: Optimal for businesses looking for enhanced security, dedicated support, and private resource management, tailored to facilitate advanced AI work.

---

## Who We Serve

Hugging Face caters to a diverse range of customers, including over **50,000 organizations** like:

- **Amazon**
- **Google**
- **Meta**
- **Microsoft**

Our offerings are utilized by non-profits, large enterprises, and individual developers, all drawn to our open-source philosophy and commitment to community collaboration.

---

## Company Culture

At Hugging Face, we believe in **community-driven development**, where everyone contributes to building the future of AI. Our culture emphasizes inclusivity, innovation, and collaboration. We encourage team members to share their ideas and projects, fostering an environment where creativity flourishes.

---

## Careers at Hugging Face

We are always on the lookout for passionate individuals to join our team. Working at Hugging Face offers the opportunity to be part of a pioneering community that is shaping the AI landscape. 

**Why Work with Us?**
- Engage in cutting-edge AI research and development.
- Collaborate with top industry experts.
- Contribute to open-source projects impacting the global AI community.

**Open Positions:** Explore current job openings on our [Careers Page](https://huggingface.co/jobs).

---

## Join Us

Are you ready to be part of the AI revolution? Whether you're an individual looking to enhance your ML skills, an organization seeking enterprise solutions, or a potential recruit desiring a dynamic career, *Hugging Face* is your gateway to the future of AI.

**Explore more at:** [Hugging Face Website](https://huggingface.co)

---

Together, let’s build the future of AI!

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>